In [494]:
import os
import pickle
import numpy as np
import math
import matplotlib.pyplot as plt
import math
import pandas as pd
from sklearn.metrics import mean_squared_error

In [495]:
a=1
b=5
n=1000 # number of sample
delta=8/255 #noise level
delta2=delta
aug=5

In [496]:




# Get the current working directory
current_dir = os.getcwd()

# Specify the name of the folder containing CIFAR-10 batches
folder_name = 'cifar10_batches'  # Replace with your actual folder name

# Construct the path to the folder
folder_path = os.path.join(current_dir, folder_name)

# Initialize empty lists to store data and labels
all_data = []
all_labels = []

# Load and concatenate training batches
for i in range(1, 6):
    batch_file_path = os.path.join(folder_path, f'data_batch_{i}')
    with open(batch_file_path, 'rb') as file:
        batch_data = pickle.load(file, encoding='bytes')
        all_data.append(batch_data[b'data'])
        all_labels.extend(batch_data[b'labels'])

# Load test batch
test_batch_file_path = os.path.join(folder_path, 'test_batch')
with open(test_batch_file_path, 'rb') as file:
    test_batch_data = pickle.load(file, encoding='bytes')
    all_data.append(test_batch_data[b'data'])
    all_labels.extend(test_batch_data[b'labels'])

# Convert the lists to NumPy arrays, cast to float, and normalize
x_train_and_test = np.concatenate(all_data, axis=0).astype(float) / 255.0
y_train_and_test = np.array(all_labels).astype(float)

# Reshape the data if needed (CIFAR-10 images are 32x32x3)
x_train_and_test = x_train_and_test.reshape((len(x_train_and_test), 32, 32, 3))

# Split into training and test sets
x_train = x_train_and_test[:-len(test_batch_data[b'labels'])]
y_train = y_train_and_test[:-len(test_batch_data[b'labels'])]
x_test = x_train_and_test[-len(test_batch_data[b'labels']):]
y_test = y_train_and_test[-len(test_batch_data[b'labels']):]

# Flatten each image in the training set
#x_train_flat = x_train.reshape((len(x_train), -1))  # -1 automatically computes the size of the remaining dimensions
 
# Print the data types and the shape of the loaded, normalized, and flattened training data
#print("Data type of x_train_flat:", x_train_flat.dtype)
#print("Normalized and flattened training data shape:", x_train_flat.shape)

In [497]:
# Choose the labels 'a' and 'b'
label_1 = a  # Replace with the actual label you want to extract
label_2 = b  # Replace with the actual label you want to extract

# Filter training set
mask_train_1 = (y_train == label_1)
mask_train_2 = (y_train == label_2)
mask_train = np.logical_or(mask_train_1, mask_train_2)

trainx = x_train[mask_train]
trainy = y_train[mask_train]

# Assign values -1 for label 'a' and 1 for label 'b'
trainy= np.where(trainy == label_1, -1, 1)

# Filter test set
mask_test_1 = (y_test == label_1)
mask_test_2 = (y_test == label_2)
mask_test = np.logical_or(mask_test_1, mask_test_2)

testx = x_test[mask_test]
testy = y_test[mask_test]

# Assign values -1 for label 'a' and 1 for label 'b'
testy = np.where(testy == label_1, -1, 1)

# Flatten each image in the extracted sets
trainx= trainx.reshape((len(trainx), -1))
testx = testx.reshape((len(testx), -1))

# Print the shapes of the extracted and flattened sets
print("Shape of x_train_ab_flat:", trainx.shape)
print("Shape of y_train_ab:", trainy.shape)
print("Shape of x_test_ab_flat:", testx.shape)
print("Shape of y_test_ab:", testy.shape)


Shape of x_train_ab_flat: (10000, 3072)
Shape of y_train_ab: (10000,)
Shape of x_test_ab_flat: (2000, 3072)
Shape of y_test_ab: (2000,)


In [498]:
# Get the indices of the samples
sample_indices = np.random.choice(len(trainx), n, replace=False)

# Extract the sampled data points
x_train_sampled = trainx[sample_indices]
y_train_sampled = trainy[sample_indices]

# Print the shapes of the sampled and flattened set
print("Shape of x_train_sampled_flat:", x_train_sampled.shape)
print("Shape of y_train_sampled:", y_train_sampled.shape)


Shape of x_train_sampled_flat: (1000, 3072)
Shape of y_train_sampled: (1000,)


In [499]:
def Ker(X,Y):
    x=np.sum(np.square(X),axis=1)
    y=np.sum(np.square(Y),axis=1)
    x=x.reshape(len(x),1)
    y=y.reshape(len(y),1)
    temp1=x.repeat(len(y),axis=1)
    temp2=y.repeat(len(x),axis=1)
    squ=temp1+np.transpose(temp2)-2*np.matmul(X,np.transpose(Y))
    #squ=np.sqrt(squ)
    mat=np.exp(-squ/200)  # kernel bandwith 1,100,200,784
    return mat

In [500]:
train1=x_train_sampled
ytrain1=y_train_sampled
test1=testx
Ytest=testy
N=np.random.normal(0,1,size=(len(test1),3072))
test=test1+delta2*N
P=np.random.normal(0,1,size=(aug,len(train1),3072))
temp2=train1+delta*P
temp2=np.array([temp2]).reshape(aug*len(train1), 3072)
    
train = temp2 #augmentation training set

In [501]:
z=[10**(-10), 10**(-8),10**(-7),10**(-6),10**(-5),10**(-4),10**(-3),10**(-2)]
error=[]
for R in z:
    print('R=',R)
    MIS=np.zeros(1)
    for a in range(1):
        #P=np.random.normal(0,1,size=(aug,len(train1),3072))
        #temp2=train1+delta*P
        #temp2=np.array([temp2]).reshape(aug*len(train1), 3072)
    
        #train = temp2 #augmentation training set
        err=np.zeros(1)
        for j in range(1):
            #N=np.random.normal(0,1,size=(len(test1),3072))
            #test=test1+delta2*N
            Ytrain=np.tile(ytrain1,aug)
            b=Ker(test,train)
            k1=Ker(train,train)
            n=len(Ytest)
            m=len(train)
            I2=np.identity(m)
        
            k2=np.matrix(k1)+m*R*I2



            k_inv=np.linalg.inv(k2)


            Y=Ytrain.reshape(len(Ytrain),1)
            temp1=np.matmul(b,k_inv)
            temp2=np.matmul(temp1,Y)

         
            y_pred1=temp2
            y_pred2=np.ravel(y_pred1)
            te = np.where(y_pred2<0, -np.ones_like(y_pred2), np.ones_like(y_pred2))
            #print(y_pred2)
            #err[j]=mean_squared_error(Ytest,y_pred2)
            mis=[]
            for i in range(len(Ytest)):
                if np.sign(temp2[i])!=np.sign(Ytest[i]):
                    mis.append(i)
            err[j]=len(mis)/n
        #print(err)
        print(np.max(err))
        MIS[a]=np.max(err)
    error.append(np.mean(MIS))
       

R= 1e-10


/var/folders/_t/4qp90hbx1xg52jv0k37yy8hw0000gn/T/ipykernel_15009/3894741130.py:9: RuntimeWarning: invalid value encountered in sqrt
  squ=np.sqrt(squ)


1.0
R= 1e-08
1.0
R= 1e-07
1.0
R= 1e-06


KeyboardInterrupt: 

In [ ]:
#no augmented error

error2=[]

for R in z:
    MIS=np.zeros(1)
    for a in range(1):
        #N=np.random.normal(0,1,size=(len(test1),3072))
        #test=test1+delta2*N
        B_1=Ker(test,train1)
        K_1=Ker(train1,train1)
        M=len(train1)
        I_2=np.identity(M)

        K_2=np.matrix(K_1)+M*R*I_2



        Kinv=np.linalg.inv(K_2)
        Y=ytrain1.reshape(len(ytrain1),1)
        temp1=np.matmul(B_1,Kinv)
        temp2=np.matmul(temp1,Y)

        y_bar=np.average(Ytest)

        ybar=np.multiply(np.ones(n),y_bar)
        num=0
        den=0
        y_pred1=temp2
        y_pred2=np.ravel(y_pred1)
        te = np.where(y_pred2<0, -np.ones_like(y_pred2), np.ones_like(y_pred2))
        mis=[]
        for i in range(len(Ytest)):
            if np.sign(temp2[i])!=np.sign(Ytest[i]):
                mis.append(i)
        
        #num = np.linalg.norm(te - Ytest)**2
        #den = np.linalg.norm(ybar-Ytest)**2
        #errors=(num/den)
        #print('misclassification=',mis)
        MIS[a]=len(mis)/n
    print(MIS)
    print(np.max(MIS))
    error2.append(np.max(MIS))

In [ ]:
plt.semilogx(z,error,label='Aug-')
plt.semilogx(z,error2,label='NoAug-')
plt.legend()
plt.show()
